# Working notebook for documents labelling
to be done:
- words cloud for each tag subset

In [71]:
import pandas as pd
import numpy as np
import os
TEXTS_PATH = os.path.join(os.path.dirname(os.getcwd()), "src", "text_extraction", "txt_selection")
RESPONSES_PATH = os.path.join(os.path.dirname(os.getcwd()), "src", "scraping", "output", "responses_2018_now.json")

pd.set_option('display.max_rows', 10)

In [72]:
#retrieving the names of all selected files
TEXTS_PATH
txt_filenames = [f for f in os.listdir(TEXTS_PATH) if f.endswith(".txt")]

In [11]:
#reading json file to retrieve tags
responses = pd.read_json(RESPONSES_PATH)
#https://raw.githubusercontent.com/naomithiru/KPMG-Usecase/philippe/src/scraping/output/responses_2018_now.json

Index(['jcId', 'jcFr', 'jcNl', 'titleFr', 'titleNl', 'themesFr', 'themesNl',
       'signatureDate', 'validityDate', 'depositDate', 'recordDate',
       'depositRegistrationDate', 'depositNumber', 'enforced',
       'royalDecreeDate', 'noticeDepositMBDate', 'publicationRoyalDecreeDate',
       'correctedDate', 'documentLink', 'documentSize', 'scopeFr', 'scopeNl',
       'noScopeFr', 'noScopeNl'],
      dtype='object')

In [79]:
#filtering only relevant columns
columns = ['jcId', 'jcFr', 'titleFr', 'themesFr', 'scopeFr', 'noScopeFr', 'documentLink']
responses2 = responses.copy(deep=True).loc[:,columns]
#getting txt_name from original pdf name
responses2["txt_name"] = responses2["documentLink"]
responses2["txt_name"] = responses2["txt_name"].str.replace("/","-")
responses2["txt_name"] = responses2["txt_name"].str.replace(".pdf","_FR.txt")
#filtering json only for selected files and columns
columns += ['txt_name']
responses2 = responses2.loc[responses2.txt_name.isin(txt_filenames), columns]
responses2.head(2)

t:\users\fra\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,jcId,jcFr,titleFr,themesFr,scopeFr,noScopeFr,documentLink,txt_name
34,1110000,COMMISSION PARITAIRE DES CONSTRUCTIONS METALLI...,modification du régime de pension sectoriel so...,[PENSIONS COMPÉMENTAIRES ET ASSURANCES GROUPES],None,[les employeurs et ouvriers des entreprises ex...,111/111-2018-013525.pdf,111-111-2018-013525_FR.txt
35,1110000,COMMISSION PARITAIRE DES CONSTRUCTIONS METALLI...,allocation spéciale compensatoire,[PRIME SYNDICALE],None,None,111/111-2018-012196.pdf,111-111-2018-012196_FR.txt


In [81]:
#generating all possible FR themes
themes_fr = []
for r in  responses2.themesFr:
    if r is not None:
        for t in r:
            if t not in themes_fr:
                themes_fr += [t]
themes_fr

['PENSIONS COMPÉMENTAIRES ET ASSURANCES GROUPES',
 'PRIME SYNDICALE',
 'TRAVAIL EN ÉQUIPE ET DE NUIT',
 'PÉCULE DE VACANCES',
 "PRIME PROPRE AU SECTEUR OU À L'ENTREPRISE",
 'REMBOURSEMENT DE FRAIS (HORS FRAIS DE DÉPLACEMENT)',
 'DÉTACHEMENT',
 "PRIME DE FIN D'ANNÉE",
 'SALAIRES',
 'JEUNES TRAVAILLEURS: SALAIRES',
 'FORMATION (EXCL. FORMATION SYNDICALE)',
 'TRAVAILLEURS AGÉS-EXCL.PENSIONS COMPL, PRÉPENSION(RCC), CRÉDIT-TEMPS',
 'MALADIE/ACCIDENT/DÉCÈS',
 'CHÔMAGE (AUTRE QUE LE CHÔMAGE ÉCONOMIQUE DES EMPLOYÉS)',
 "RÉGIME DE CHÔMAGE AVEC COMPLÉMENT D'ENTREPRISE (RCC)",
 'FORMATION SYNDICALE',
 "FONDS DE SÉCURITÉ D'EXISTENCE",
 'INDEMNITÉS POUR FRAIS DE DÉPLACEMENT',
 'CRÉDIT-TEMPS/DIMINUTION DE CARRIÈRE, EMPLOI FIN DE CARRIÈRE',
 'GROUPES À RISQUE',
 'TRAVAIL TEMPORAIRE ET TRAVAIL INTÉRIMAIRE',
 'RECRUTEMENT',
 'BIEN-ÊTRE AU TRAVAIL',
 'PRÉAVIS/LICENCIEMENT',
 'TEMPS DE TRAVAIL EN HEURES',
 'CHÈQUES-REPAS',
 'ECOCHÈQUES',
 'JOUR DE CONGÉ PAYÉ (PAS DE JOUR RTT OU DE COMPENSATION) ET JOUR F

In [154]:
#code to retrieve the entire text body
#removing for testing purpose
if "chapters" in responses2.columns.to_list():
    responses2.drop(columns=["chapters"], inplace=True)
#if 2 > 1:
#
chapters = []

for filename in responses2.txt_name.to_list():
    filename = responses2.txt_name.to_list()[10]
    file_path = os.path.join(TEXTS_PATH, filename)
    with open(file_path, 'r', encoding="utf8") as f:
        #to retrieve entire body not necessary
        #responses2.loc[responses2.txt_name == filename, "txt_body"] = f.read()

        line_list = f.readlines()
        whole_text = ""
        for line in line_list:
            whole_text += line
        ch_list = whole_text.lower().split("chapitre")
        if len(ch_list) > 1:
            print(len(ch_list))
        chapters += [ch_list]

#responses2.loc[responses2.txt_name == filename, "chapters"].values = np.array(ch_list)
responses2["chapters"] = chapters

In [153]:
for r in responses2.chapters.to_list():
    print(len(r))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [119]:
type(responses2.loc[responses2.txt_name == filename, 'themesFr'].values)

numpy.ndarray

In [142]:
responses2.loc[responses2.txt_name == filename, "chapters"] = np.array(ch_list)

In [143]:
responses2.loc[responses2.txt_name == filename, "chapters"].values

array(["neerlegging-dépôt: 21/02/2018 regist.-enregistr.: 05/03/2018 n°: 144977/c0/111\n\ncommission paritaire 111 constructions metallique, mecanique et electrique\n\nconvention collective de travail du 19 février 2018\n\ntravail faisable modifications de carriere\n\n\\rt. 1 — champ d'application\n\na présente convention collective de travail s'applique ux employeurs et aux ouvriers des entreprises essortissant à la commission paritaire des onstructions métallique, mécanique et électrique.\n\n)n entend par “ouvriers”, les ouvriers et les ouvrières.\n\n\\rt. 2 — objet\n\n\\ partir du 1° janvier 2018, cette convention collective ccorde des indemnités aux ouvriers qui ont atteint âge de 58 ans au moins, dans le cadre des mesures isant à diminuer la charge de travail.\n\n‚es indemnités sont payées par le fonds de sécurité 'existence des fabrications métalliques, ci-après ommé le fsefm.\n\na présente convention collective de travail a été onclue en application de la convention collective d

In [83]:
#testing code for filter depending on theme fr
#for t in themes:
t = themes_fr[0]
mask = responses2.themesFr.apply(lambda x: t in x)
r_t = responses2[mask]
print(t)
print(r_t)

ValueError: Must have equal len keys and value when setting with an iterable

# NLP to be implemented

In [ ]:
from src.information_extraction.nlp_preprocess import Preprocess
preprocess = Preprocess(nlp_model='en_core_web_md')

t = preprocess.clean_text(text_sample)

In [ ]:
responses2.loc[:, "txt_cleaned"] =  responses2.loc[:, "txt_body"].apply(preprocess.clean_text)
